In [1]:
from dotenv import load_dotenv
import os
import base64
from requests import post,get
import json
import requests
import pandas as pd

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    result= get(query_url, headers = headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists")
        return None
    
    return json_result[0]

def get_songs_recommendations(token, artist_id, min_popularity, max_popularity):
    url = f"https://api.spotify.com/v1/recommendations?limit=100&market=US&seed_artists={artist_id}&min_popularity={min_popularity}&max_popularity={max_popularity}"
    headers = get_auth_header(token)
    result = get(url, headers= headers)
    if result.status_code ==200:
        json_result=json.loads(result.content)
        return json_result['tracks']
    else:
        print(f"Failed to fetch recommendations. Status code: {result.status_code}")
        return None

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def get_song_audio_features(token, song_id):
    url = f"https://api.spotify.com/v1/audio-features?ids={','.join(song_id)}"
    headers = get_auth_header(token)
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        json_result = response.json()  # Use .json() method to parse the JSON data
        return json_result
    else:
        print(f"Failed to fetch audio features. Status code: {response.status_code}")
        return None

In [2]:
recommended_songs_df = pd.read_csv('recommended_song.csv')

In [3]:
recommended_songs_id = recommended_songs_df['id']

In [4]:
recommended_songs_id

0         0FcjlhzxLTUnNseW0aNjrg
1         6GgjPqWh25IVIPNTp1soPd
2         30XUZHuO3Zn79Kpe5ylkXI
3         6y7mf352StjAm4bMDkeMyH
4         4HLElWheO0EfBtWQFkeOcG
                   ...          
237678    4cQEUG90GcPK9pPoTSmDBX
237679    4EkOA5QmlVDoq5JtSQMRxc
237680    2ABUsYOPTFDQzkjcY4gMd1
237681    361gBlZ84MJlnXfsxVXyuE
237682    7BRZB16KZyGQbf8jsN9Mdn
Name: id, Length: 237683, dtype: object

In [5]:
recommended_songs_id1 = recommended_songs_id[0:80000]
recommended_songs_id2 = recommended_songs_id[80000:160000]
recommended_songs_id3 = recommended_songs_id[160000:237683]


In [6]:
import time
token = get_token()
recommended_features = []
for group in chunker(recommended_songs_id, 100):
    # time.sleep(3)
    recommended_feature_group = get_song_audio_features(token,group)
    recommended_features.append(recommended_feature_group)

In [7]:
df_recommended_audio_features = pd.DataFrame(recommended_features[0]['audio_features'])

In [8]:
recommended_features[1]['audio_features'][0]

{'danceability': 0.888,
 'energy': 0.515,
 'key': 1,
 'loudness': -6.246,
 'mode': 1,
 'speechiness': 0.271,
 'acousticness': 0.247,
 'instrumentalness': 0,
 'liveness': 0.446,
 'valence': 0.264,
 'tempo': 141.009,
 'type': 'audio_features',
 'id': '5S1IUPueD0xE0vj4zU3nSf',
 'uri': 'spotify:track:5S1IUPueD0xE0vj4zU3nSf',
 'track_href': 'https://api.spotify.com/v1/tracks/5S1IUPueD0xE0vj4zU3nSf',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5S1IUPueD0xE0vj4zU3nSf',
 'duration_ms': 225893,
 'time_signature': 4}

In [9]:
df_recommended_audio_features = pd.DataFrame(recommended_features[0]['audio_features'])
for x in range(1, len(recommended_features)):
    for y in range(0,100):
        if recommended_features[x]['audio_features'][y]:
            temp_df = pd.DataFrame([recommended_features[x]['audio_features'][y]])
            df_recommended_audio_features = pd.concat([df_recommended_audio_features, temp_df]).reset_index(drop=True)
        else:
            break



IndexError: list index out of range

In [10]:
df_recommended_audio_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,0
0,0.818,0.837,9.0,-5.915,1.0,0.0376,0.00956,0.000412,0.0582,0.737,125.010,audio_features,0FcjlhzxLTUnNseW0aNjrg,spotify:track:0FcjlhzxLTUnNseW0aNjrg,https://api.spotify.com/v1/tracks/0FcjlhzxLTUn...,https://api.spotify.com/v1/audio-analysis/0Fcj...,201360.0,4.0,NaN
1,0.426,0.713,7.0,-5.328,1.0,0.3600,0.66600,0.000000,0.2400,0.641,77.827,audio_features,6GgjPqWh25IVIPNTp1soPd,spotify:track:6GgjPqWh25IVIPNTp1soPd,https://api.spotify.com/v1/tracks/6GgjPqWh25IV...,https://api.spotify.com/v1/audio-analysis/6Ggj...,213814.0,4.0,NaN
2,0.611,0.284,2.0,-19.339,1.0,0.0800,0.79300,0.000006,0.1470,0.712,92.342,audio_features,30XUZHuO3Zn79Kpe5ylkXI,spotify:track:30XUZHuO3Zn79Kpe5ylkXI,https://api.spotify.com/v1/tracks/30XUZHuO3Zn7...,https://api.spotify.com/v1/audio-analysis/30XU...,181368.0,4.0,NaN
3,0.823,0.704,1.0,-4.740,1.0,0.1460,0.02190,0.000000,0.1160,0.114,135.012,audio_features,6y7mf352StjAm4bMDkeMyH,spotify:track:6y7mf352StjAm4bMDkeMyH,https://api.spotify.com/v1/tracks/6y7mf352StjA...,https://api.spotify.com/v1/audio-analysis/6y7m...,261992.0,4.0,NaN
4,0.703,0.540,9.0,-7.336,1.0,0.1980,0.20200,0.000000,0.1830,0.296,119.988,audio_features,4HLElWheO0EfBtWQFkeOcG,spotify:track:4HLElWheO0EfBtWQFkeOcG,https://api.spotify.com/v1/tracks/4HLElWheO0Ef...,https://api.spotify.com/v1/audio-analysis/4HLE...,345960.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237678,0.728,0.339,11.0,-9.890,0.0,0.2140,0.31800,0.000000,0.1120,0.574,138.015,audio_features,4cQEUG90GcPK9pPoTSmDBX,spotify:track:4cQEUG90GcPK9pPoTSmDBX,https://api.spotify.com/v1/tracks/4cQEUG90GcPK...,https://api.spotify.com/v1/audio-analysis/4cQE...,163585.0,4.0,NaN
237679,0.469,0.685,11.0,-9.643,0.0,0.0374,0.00037,0.911000,0.0998,0.559,160.086,audio_features,4EkOA5QmlVDoq5JtSQMRxc,spotify:track:4EkOA5QmlVDoq5JtSQMRxc,https://api.spotify.com/v1/tracks/4EkOA5QmlVDo...,https://api.spotify.com/v1/audio-analysis/4EkO...,77298.0,4.0,NaN
237680,0.419,0.953,0.0,-5.524,1.0,0.0502,0.05100,0.000007,0.3500,0.683,171.074,audio_features,2ABUsYOPTFDQzkjcY4gMd1,spotify:track:2ABUsYOPTFDQzkjcY4gMd1,https://api.spotify.com/v1/tracks/2ABUsYOPTFDQ...,https://api.spotify.com/v1/audio-analysis/2ABU...,157193.0,4.0,NaN
237681,0.698,0.651,0.0,-9.147,1.0,0.0793,0.05000,0.000000,0.1160,0.673,143.937,audio_features,361gBlZ84MJlnXfsxVXyuE,spotify:track:361gBlZ84MJlnXfsxVXyuE,https://api.spotify.com/v1/tracks/361gBlZ84MJl...,https://api.spotify.com/v1/audio-analysis/361g...,143333.0,4.0,NaN


In [11]:
df_recommended_songs_final = pd.merge(recommended_songs_df, df_recommended_audio_features, on = 'id')

In [12]:
df_recommended_songs_final=df_recommended_songs_final.drop_duplicates('name')
df_recommended_songs_final = df_recommended_songs_final.drop(['Unnamed: 0',0], axis =1)
df_recommended_songs_final = df_recommended_songs_final.dropna()
df_recommended_songs_final['duration_s'] = df_recommended_songs_final['duration_ms'] / 1000

In [13]:
df_recommended_songs_final

,name,artist,id,popularity,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,duration_s
0,Dont Worry (its on me) - Radio Edit,Sir William,0FcjlhzxLTUnNseW0aNjrg,24,0.818,0.837,9.0,-5.915,1.0,0.0376,...,0.0582,0.737,125.010,audio_features,spotify:track:0FcjlhzxLTUnNseW0aNjrg,https://api.spotify.com/v1/tracks/0FcjlhzxLTUn...,https://api.spotify.com/v1/audio-analysis/0Fcj...,201360.0,4.0,201.360
1,Slow It Down,Short Dawg,6GgjPqWh25IVIPNTp1soPd,32,0.426,0.713,7.0,-5.328,1.0,0.3600,...,0.2400,0.641,77.827,audio_features,spotify:track:6GgjPqWh25IVIPNTp1soPd,https://api.spotify.com/v1/tracks/6GgjPqWh25IV...,https://api.spotify.com/v1/audio-analysis/6Ggj...,213814.0,4.0,213.814
2,Club Paradise - Remix,Kigity K,30XUZHuO3Zn79Kpe5ylkXI,21,0.611,0.284,2.0,-19.339,1.0,0.0800,...,0.1470,0.712,92.342,audio_features,spotify:track:30XUZHuO3Zn79Kpe5ylkXI,https://api.spotify.com/v1/tracks/30XUZHuO3Zn7...,https://api.spotify.com/v1/audio-analysis/30XU...,181368.0,4.0,181.368
3,"Chanel Vintage (feat. Future, Young Thug)",Metro Boomin,6y7mf352StjAm4bMDkeMyH,31,0.823,0.704,1.0,-4.740,1.0,0.1460,...,0.1160,0.114,135.012,audio_features,spotify:track:6y7mf352StjAm4bMDkeMyH,https://api.spotify.com/v1/tracks/6y7mf352StjA...,https://api.spotify.com/v1/audio-analysis/6y7m...,261992.0,4.0,261.992
4,Dedicated,The Game,4HLElWheO0EfBtWQFkeOcG,31,0.703,0.540,9.0,-7.336,1.0,0.1980,...,0.1830,0.296,119.988,audio_features,spotify:track:4HLElWheO0EfBtWQFkeOcG,https://api.spotify.com/v1/tracks/4HLElWheO0Ef...,https://api.spotify.com/v1/audio-analysis/4HLE...,345960.0,4.0,345.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237647,Finesse Gawd,Bryce Oliver,4cQEUG90GcPK9pPoTSmDBX,38,0.728,0.339,11.0,-9.890,0.0,0.2140,...,0.1120,0.574,138.015,audio_features,spotify:track:4cQEUG90GcPK9pPoTSmDBX,https://api.spotify.com/v1/tracks/4cQEUG90GcPK...,https://api.spotify.com/v1/audio-analysis/4cQE...,163585.0,4.0,163.585
237648,Giving My All 2 U,WEB,4EkOA5QmlVDoq5JtSQMRxc,62,0.469,0.685,11.0,-9.643,0.0,0.0374,...,0.0998,0.559,160.086,audio_features,spotify:track:4EkOA5QmlVDoq5JtSQMRxc,https://api.spotify.com/v1/tracks/4EkOA5QmlVDo...,https://api.spotify.com/v1/audio-analysis/4EkO...,77298.0,4.0,77.298
237649,I MISS 2007,poptropicaslutz!,2ABUsYOPTFDQzkjcY4gMd1,48,0.419,0.953,0.0,-5.524,1.0,0.0502,...,0.3500,0.683,171.074,audio_features,spotify:track:2ABUsYOPTFDQzkjcY4gMd1,https://api.spotify.com/v1/tracks/2ABUsYOPTFDQ...,https://api.spotify.com/v1/audio-analysis/2ABU...,157193.0,4.0,157.193
237650,drugs you should try,Deep October,361gBlZ84MJlnXfsxVXyuE,35,0.698,0.651,0.0,-9.147,1.0,0.0793,...,0.1160,0.673,143.937,audio_features,spotify:track:361gBlZ84MJlnXfsxVXyuE,https://api.spotify.com/v1/tracks/361gBlZ84MJl...,https://api.spotify.com/v1/audio-analysis/361g...,143333.0,4.0,143.333


In [14]:
df_recommended_songs_final.to_csv('top_recommendations_features.csv')